In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
import numpy as np
import time
from tensorflow import keras
import psutil
import copy
device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Flatten the images
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# Convert to tensors
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).long().to(device)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_test_tensor = torch.from_numpy(y_test).long().to(device)

# Create the training dataset
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Define the size of the validation set (e.g., 20% of the training set)
val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size

# Split the dataset into training and validation datasets
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # No shuffling for validation
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

<ipython-input-241-8f83f1f3566b>:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  x_train_tensor = torch.from_numpy(x_train).float().to(device)


In [ ]:
class MLP_Mnist(nn.Module):
    def __init__(self):
        super(MLP_Mnist, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, x):
        out = self.relu1(self.fc1(x))
        out = self.relu2(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
# UNFINISHED! ITS STILL NOT WHAT I WANT IT TO BE! I STILL DONT GET IT!

class PCGMFTrainer:
    def __init__(self, model, pcriterion, poptimizer, perturbation_magnitude, patience=5, delta=0):
        self.model = model
        self.criterion = pcriterion
        self.optimizer = poptimizer
        self.perturbation_magnitude = perturbation_magnitude
        self.saved_models = []  # Store model states and their corresponding losses
        self.early_stopping = EarlyStopping(patience, delta)
        self.patience = patience
        self.delta = delta
        self.loss_global = 0

    def train(self, train_loader, val_loader, epochs, perturbation_index):
        self.early_stopping = EarlyStopping(self.patience, self.delta)
        t1 = time.time()
        self.model.train()  # Set the model to training mode
        best_val_loss = float('inf')  # Track the best validation loss

        for epoch in range(epochs):
            running_loss = 0.0
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.view(data.size(0), -1).to(device), target.to(device)  # Flatten the images
                self.optimizer.zero_grad()  # Zero the parameter gradients
                outputs = self.model(data)  # Forward pass
                loss = self.criterion(outputs, target)  # Calculate loss
                loss.backward()  # Backward pass
                self.optimizer.step()  # Optimize weights
                running_loss += loss.item()  # Accumulate loss

            avg_loss = running_loss / len(train_loader)  # Calculate average loss for the epoch
            val_loss = calculate_validation_loss(self.model, val_loader, self.criterion)  # Validate

            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, Validation Loss: {val_loss:.4f}')

            # Check for early stopping
            if self.early_stopping(val_loss):
                self.save_model(calculate_accuracy(model, val_loader))
                print("Early stopping triggered.")
                break

        t2 = time.time()
        print(f"training time for pertubation {perturbation_index}: {t2 - t1:.2f}s")

    def save_model(self, accuracy):
        # Save the model state and its corresponding loss
        self.saved_models.append((copy.deepcopy(model), accuracy))

    def apply_perturbation(self):
        with torch.no_grad():
            for param in self.model.parameters():
                noise = (torch.rand_like(param) * 2 - 1) * self.perturbation_magnitude
                param.add_(noise)

    def train_with_perturbations(self, train_loader, val_loader, epochs, perturbations):
        # Train normally first and capture the final loss
        t1 = time.time()
        self.train(train_loader, val_loader, epochs, 0)

        for _ in range(perturbations):
            self.apply_perturbation()  # Apply perturbation to model parameters
            self.train(train_loader, val_loader, epochs, _+1)  # Continue training with perturbations
        t2 = time.time()
        print(f"total time = {t2-t1}")

    def evaluate(self):
        best_accuracy = 0
        iteration = 0
        best_accuracy_index = None
        if not self.saved_models:
            print("No saved models to evaluate.")
            return
        for i in PCGMF.saved_models:
          if i[1]>best_accuracy:
            best_accuracy = i[1]
            best_accuracy_index = iteration
          iteration += 1
        print(f"Best model is index {best_accuracy_index} with validation accuracy {best_accuracy}")
        print(f"Best model's test accuracy : {calculate_accuracy(PCGMF.saved_models[best_accuracy_index][0], test_loader)}")


In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True  # Indicate convergence
        return False  # Continue training

In [ ]:
def calculate_validation_loss(model, data_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for data, target in data_loader:
            data = data.view(data.size(0), -1).to(device)  # Flatten the images
            target = target.to(device)

            # Forward pass
            outputs = model(data)

            # Calculate loss
            loss = criterion(outputs, target)

            # Accumulate loss
            total_loss += loss.item() * data.size(0)  # Multiply by batch size to get total loss
            total_samples += data.size(0)  # Count total samples

    average_loss = total_loss / total_samples  # Average loss over all samples
    return average_loss

In [ ]:
def calculate_accuracy(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for data, target in data_loader:
            data = data.view(data.size(0), -1).to(device)  # Flatten the images
            target = target.to(device)

            # Forward pass
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)  # Get the class with the highest probability

            # Update correct and total counts
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total  # Calculate accuracy as a percentage
    return accuracy

In [ ]:
model = MLP_Mnist().to(device)
#loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
early_stopping = EarlyStopping(patience=5, delta=0.001)

PCGMF = PCGMFTrainer(model, criterion, optimizer, perturbation_magnitude=0.01)

In [ ]:
PCGMF.train_with_perturbations(train_loader, epochs=100, perturbations=3, val_loader=val_loader)

Epoch [1/100], Loss: 0.7984, Validation Loss: 0.5033
Epoch [2/100], Loss: 0.4823, Validation Loss: 0.4536
Epoch [3/100], Loss: 0.4341, Validation Loss: 0.4267
Epoch [4/100], Loss: 0.4029, Validation Loss: 0.4075
Epoch [5/100], Loss: 0.3811, Validation Loss: 0.3886
Epoch [6/100], Loss: 0.3637, Validation Loss: 0.4157
Epoch [7/100], Loss: 0.3485, Validation Loss: 0.3777
Epoch [8/100], Loss: 0.3368, Validation Loss: 0.3819
Epoch [9/100], Loss: 0.3253, Validation Loss: 0.3686
Epoch [10/100], Loss: 0.3161, Validation Loss: 0.3778
Epoch [11/100], Loss: 0.3086, Validation Loss: 0.3722
Epoch [12/100], Loss: 0.3003, Validation Loss: 0.3530
Epoch [13/100], Loss: 0.2938, Validation Loss: 0.3520
Epoch [14/100], Loss: 0.2874, Validation Loss: 0.3629
Epoch [15/100], Loss: 0.2805, Validation Loss: 0.3562
Epoch [16/100], Loss: 0.2769, Validation Loss: 0.3556
Epoch [17/100], Loss: 0.2704, Validation Loss: 0.3508
Epoch [18/100], Loss: 0.2663, Validation Loss: 0.3473
Epoch [19/100], Loss: 0.2593, Validat

In [ ]:
for i in PCGMF.saved_models:
  print(i[1])

87.925
87.51666666666667
88.2
87.99166666666666


In [ ]:
PCGMF.evaluate()

Best model is index 2 with validation accuracy 88.2
Best model's test accuracy : 86.9


In [ ]:
calculate_accuracy(PCGMF.saved_models[3][0], test_loader)

87.0